In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
import sys
import json
from DB_setup import Clone_sequence, DBD, Experiment, PFM, TF_info, Base, DBD_Clone_Maps, Protein_clone_map
from sqlalchemy.orm import sessionmaker


#setting up database directory, database will be named tf.db for now
db_directory = 'C:\\Users\\Alex\\Documents\\fordyce_rotation\\tf5.db'

#setting up engine to connect to the database, using sqlite
engine = create_engine('sqlite:///'+db_directory)

Base.metadata.create_all(engine)


#JSON files with the information
jaspar_file, uniprobe_file = ['jaspar_success.json', 'uniprobe_final.json']

with open(jaspar_file ,'r') as jasp:
    jaspar = json.loads(jasp.read())

with open(uniprobe_file ,'r') as uni:
    uniprobe = json.loads(uni.read())

import re
protein_seqs = []
experiments = []
pfms = []
protein_info = []
dbds = []

Session = sessionmaker(bind=engine)
session = Session()

#putting in jaspar rows first
for tf, info in jaspar.items():
    protein_name = '_'.join(tf.split('::'))
    sequence = info['sequence']
    if not sequence:
        sequence = None
    
    # making Clone_sequence object, which corresponds to a row
    protein_seq_to_add = Clone_sequence(protein_name, sequence)
    uniprot_ids = info['acc']
    for uniprot_id in uniprot_ids:
        marker = 0
        #searching through the already created Protein_information objects in this session to see
        #if one with the same uniprot ID has already been made, and adding the link between the 
        #clone_sequence and the protein.
        for obj in session.new:
            if type(obj) == TF_info:
                #if the protein is already in the session, just adding links
                if obj.uniprot == uniprot_id:
                    marker = 1
                    #creating a link between the protein and the clone sequence
                    #the links will have their own table, since the relationship is
                    #many-to-many, due to complexes and mutants
                    clone_protein_link = Protein_clone_map(protein_seq_to_add, obj)
                    
                    #adding link object to the session so it's now pending
                    session.add(clone_protein_link)
                    
                    #adding the link as a relationship attribute to the current clone
                    protein_seq_to_add.clone_protein_map.append(clone_protein_link)
                    
                    
                    #adding the link as a relationship attribute to the protein
                    obj.clone_protein_map.append(clone_protein_link)
        
        #if the uniprot ID has not already been added to the session, making a new
        #protein information object, and adding in the links as above
        if not marker:
            protein_infor = TF_info(tf_class = info['class'], family = info['family'],\
                                species = str(info['species']), uniprot = uniprot_id)
            
            clone_protein_link = Protein_clone_map(protein_seq_to_add, protein_infor)
            session.add(clone_protein_link)
            protein_seq_to_add.clone_protein_map.append(clone_protein_link)
            protein_infor.clone_protein_map.append(clone_protein_link)
            session.add(protein_infor)
            
    #adding in the experiment rows
    exp = Experiment(pubmed_id = info['pubmed'], assay = info['type'])
    
    #linking the clones to the experiments (1:M), because one clone can be tested multiple times,
    #but an experiment in this context is defined to be testing one protein (which allows for multiple
    #different types of assays in one pubmed ID, which may be unrealistic)
    protein_seq_to_add.experiments = [exp]
    
    #adding in the dna binding domains, and their links to the clone sequences
    if info['dbds']:
        for dbd in info['dbds']:
            if 'domain_sequence' in dbd:
                dbd_seq = dbd['domain_sequence']
            else:
                dbd_seq = None
            
            #creating DBD
            dbd_to_add = DBD(dbd_seq, dbd['dbd_type'])
            #creating DBD-clone link
            dbd_link = DBD_Clone_Maps(protein_seq_to_add, dbd_to_add)
            #adding the link to the session
            session.add(dbd_link)
            #adding the link to the dbd as a relationship to the clone sequence
            protein_seq_to_add.clone_dbd_maps.append(dbd_link)
            #adding the link to the dbd as a relationship to the dbd
            dbd_to_add.clone_dbd_maps.append(dbd_link)
            #adding the dbd to the session
            session.add(dbd_to_add)

   
    #dealing with pfms with some sick regex 
    #should probably do this with biopython Motifs
    #but this works
    A, C, G, T, _ = info['motif'].split('\n')
    A = re.findall(r'[\d]+.[\d]+', A)
    C = re.findall(r'[\d]+.[\d]+', C)
    G = re.findall(r'[\d]+.[\d]+', G)
    T = re.findall(r'[\d]+.[\d]+', T)
    for pos in range(len(A)):
        pfmpos = PFM(pos, float(A[pos]),\
                        float(C[pos]), float(G[pos]), float(T[pos]), db = 'jaspar')
        #adding in link from experiment to pfm (1:1 in real life, but 1:M in the database because of how the 
        #pfms are represented)
        exp.pfms.append(pfmpos)
        session.add(pfmpos)
        
    #adding everything else to the session
    session.add(protein_seq_to_add)
    session.add(exp)

#now on to uniprobe    
for protein in uniprobe.values():
    #until next comment, this is all pretty much the same as the jaspar code above 
    #with a different dictionary structure
    protein_name = protein['protein_mut']
    sequence = protein['sequence']
    if not sequence:
        sequence = None
    protein_seq_to_add = Clone_sequence(protein_name, sequence)
    
    uniprot_id = protein['uniprot']
    tf_class = None
    family = None
    species = protein['species']
    pubmed = protein['pubmed']
    marker = 0
    for obj in session.new:
        if type(obj) == TF_info:
            if obj.uniprot == uniprot_id:
                marker = 1
                clone_protein_link = Protein_clone_map(protein_seq_to_add, obj)
        
                session.add(clone_protein_link)
                
                protein_seq_to_add.clone_protein_map.append(clone_protein_link)
        
                obj.clone_protein_map.append(clone_protein_link)
        
            
    if not marker:
        protein_infor = TF_info(tf_class = tf_class, family = family,\
                                species = species, uniprot = uniprot_id)
        
        clone_protein_link = Protein_clone_map(protein_seq_to_add, protein_infor)
        session.add(clone_protein_link)
        protein_seq_to_add.clone_protein_map.append(clone_protein_link)
        protein_infor.clone_protein_map.append(clone_protein_link)
        session.add(protein_infor)
    #everything in uniprobe is a PBM 
    exp = Experiment(pubmed_id = pubmed, assay = 'PBM')
    
    protein_seq_to_add.experiments.append(exp)
    pwm = protein['pwm']
    #I was dumb and have some blank strings as sequences/dbd_types, so changing those to NULLs for SQL ease
    for dbd in protein['dbds']:
        if 'domain_sequence' in dbd:
            dbd_seq = dbd['domain_sequence']
        else:
            dbd_seq =  None
        if dbd_seq == '':
            dbd_seq = None
        if dbd['dbd_type']:
            dbd_type = dbd['dbd_type']
        else:
            dbd_type = None
        dbd_to_add = DBD(dbd_seq, dbd['dbd_type'])
        dbd_link = DBD_Clone_Maps(protein_seq_to_add, dbd_to_add)
        session.add(dbd_link)
        protein_seq_to_add.clone_dbd_maps.append(dbd_link)
        dbd_to_add.clone_dbd_maps.append(dbd_link)
        session.add(dbd_to_add)
    
    for pos in range(len(pwm['A'])):
        #much more nicely arranged ppms, potentially not great though
        A = pwm['A']
        C = pwm['C']
        G = pwm['G']
        T = pwm['T']
        pfmpos = PFM(pos,float(A[pos]),\
                        float(C[pos]), float(G[pos]), float(T[pos]), db = 'uniprobe')
        exp.pfms.append(pfmpos)
        session.add(pfmpos)
    session.add(protein_seq_to_add)
    session.add(exp)

session.commit()

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: dbd.DBD_sequence [SQL: 'INSERT INTO dbd ("DBD_sequence", dbd_type) VALUES (?, ?)'] [parameters: ('LARENHSEIERRRRNKMTAYITELSDMVPTCSALARKPDKLTILRMAVSHMKS', 'bHLH')]